In [1]:
from app.dotenv import base_dir, data_dir
import os
from sentence_transformers import SentenceTransformer

pdf_path = f"{data_dir}/{os.getenv('pdf_file')}.pdf"
print('pdf_path:',pdf_path )


embedding_model_name = os.getenv('embedding_model_name')
print('embeding model:',embedding_model_name )
embedding_model = SentenceTransformer(embedding_model_name)  # You can choose any available model

llm_model_name = os.getenv('llm_model_name')
print('llm model:',llm_model_name )

bert_model_name = os.getenv('bert_model_name')
print('bert model:',bert_model_name )
from app.metrics import loadBertModel
tokenizer, model, scorer = loadBertModel(bert_model_name)




pdf_path: /Users/yishai/Desktop/InsuranceHebrewRAG-Assistant/data/כתב שירות רפואה משלימה.pdf
embeding model: imvladikon/sentence-transformers-alephbert
llm model: aminadaven/dictalm2.0-instruct:f16
bert model: xlm-roberta-base


load the hebrew pdf data and format it

In [2]:
from app.pdf_reader import read_heb_pdf
full_text = read_heb_pdf(pdf_path)
print('preview of loaded text: \n',full_text[:500])

preview of loaded text: 
 תמצית תנאי הביטוח-כתב שירות-רפואה משלימה 3063()
תמצית פרטי הפוליסה
שם הביטוח כתב שירות-רפואה משלימה
סוג הביטוח כתב שירות
תקופת הביטוח מתחדשת כל שנתיים.ראה סעיף 10 לכתב השירות
תיאור הביטוח שירותי רפואה משלימה ופיזיותרפיה
הפוליסה אינה מכסה את המבוטח
ראה סעיף 6 לכתב השירות
במקרים הבאים (חריגים בפוליסה)
אחרי כמה זמן מתחילת הביטוח
90 ימים.ראה סעיף 17.2 לכתב השירות
ניתן לתבוע ולקבל תגמול (אכשרה)1
קבלת השירות מנותן שירות שבהסכם-
סך של 47 ₪ בעבור כל טיפול ו/או התייעצות שיבוצעו במרפאת נותן השירות שבהסכם.


Create RAG DB
1. split text into chunks (using RecursiveCharacterTextSplitter)
2. create rag using faiss (since its a single document  - rag DB is in-memory)

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from app.rag import *

# Create an instance of RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

# Split the text
chunks = text_splitter.split_text(full_text)
# print(f'data is split into {len(chunks)} chunks' )


rag_index = create_RAG(chunks, embedding_model)
# Print the resulting chunks
print('preview of chunked text:') 
for chunk in chunks [:5]:
    print('_'*50)
    print(chunk)

preview of chunked text:
__________________________________________________
תמצית תנאי הביטוח-כתב שירות-רפואה משלימה 3063()
תמצית פרטי הפוליסה
שם הביטוח כתב שירות-רפואה משלימה
__________________________________________________
סוג הביטוח כתב שירות
תקופת הביטוח מתחדשת כל שנתיים.ראה סעיף 10 לכתב השירות
__________________________________________________
תיאור הביטוח שירותי רפואה משלימה ופיזיותרפיה
הפוליסה אינה מכסה את המבוטח
ראה סעיף 6 לכתב השירות
__________________________________________________
במקרים הבאים (חריגים בפוליסה)
אחרי כמה זמן מתחילת הביטוח
90 ימים.ראה סעיף 17.2 לכתב השירות
__________________________________________________
ניתן לתבוע ולקבל תגמול (אכשרה)1
קבלת השירות מנותן שירות שבהסכם-


get answer to user query (free text):
1. retrieve relevant data from rag
2. create a prompt with question and context and chain of thoughts
3. recieve answer from the llm model (ollama api)
4. display user query and llm response

In [4]:
from app.llm import print_prompt_and_response
user_queries = [
                "ממתי ניתן לקבל החזר על הטיפול באקופונקטורה",
                "האם מקבלים החזר מלא על הטיפולים",
               "כמה טיפולים מכוסים",
                ]

queries_data = []
for user_query in user_queries:
    retrieved_chunks = retrieve_context(user_query, chunks, embedding_model, rag_index, print_rag_results=False, k_chunks=5)
    single_query_relevnt_data = print_prompt_and_response(user_query, retrieved_chunks, model = llm_model_name,print_rag_results=True)
    queries_data.append(single_query_relevnt_data)# used for bertsimilarity
    



===== User Input Query =====
ממתי ניתן לקבל החזר על הטיפול באקופונקטורה
===== Relevant RAG Data =====
שיטת טווינה,
משלימה שיפוי בסך של 50% מעלות הטיפול ששולמה
עיסוי רפואי,
--------------------------------------------------
ניתן לתבוע ולקבל תגמול (אכשרה)1
קבלת השירות מנותן שירות שבהסכם-
--------------------------------------------------
ביופידבק,
טיפול נטורופתי,
קבלת השירות מנותן שירות שבהסכם-
ייעוץ דיאטטי,
--------------------------------------------------
שיפוי בסך של 50% מעלות הטיפול ששולמה בפועל אך לא יותר מ-140 ₪ לכל טיפול.
--------------------------------------------------
תיאור הביטוח שירותי רפואה משלימה ופיזיותרפיה
הפוליסה אינה מכסה את המבוטח
ראה סעיף 6 לכתב השירות
--------------------------------------------------

===== Final LLM Response =====
 זכאותך לקבלת החזר עבור טיפול בדיקור סיני תלויה באקופונקטורה מסופקת על ידי נותן שירות בהסכם, עם שיפוי של 50% מעלות הטיפול בפועל (עד 140 ש"ח לכל טיפול). כדי לממש את ההחזר הזה, עליך לקבל את השירות מנותן שירות ברשימת הספקים המופיעים בתנאי 

free text query

change query to what ever you want to ask

In [5]:
# from app.llm import print_prompt_and_response

# user_query = "האם מקבלים החזר מלא על הטיפולים"
# retrieved_chunks = retrieve_context(user_query, chunks, embedding_model, rag_index, print_rag_results=False, k_chunks=5)
# single_query_relevnt_data = print_prompt_and_response(user_query, retrieved_chunks, model = llm_model_name,print_rag_results=True)

bert-score:
1. check similarity betwen user query and answer
2. check similarity between answer and expected answer in validation dataset (both compare to correct answer and wrong answer)

In [6]:

from app.metrics import bertSimilarity,evaluateTask
for single_query_relevnt_data in queries_data:
    candidate,reference = single_query_relevnt_data['llm_response'],single_query_relevnt_data['user_query']
    precision, recall, f1, similarity = bertSimilarity(candidate,reference, tokenizer, model, scorer)
    evaluateTask(precision, recall, f1, similarity,task_type=1)


Similarity between LLM response and user query
BERTScore (Precision: 0.8141), Recall: 0.8957, F1: 0.8529, Similarity: 0.9948
Similarity between LLM response and user query
BERTScore (Precision: 0.7975), Recall: 0.8992, F1: 0.8453, Similarity: 0.9927
Similarity between LLM response and user query
BERTScore (Precision: 0.8084), Recall: 0.8794, F1: 0.8424, Similarity: 0.9935


check on validation dataset

In [7]:


import json
import time

# Function to load questions from a JSON file
def load_validation_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

val_data = load_validation_dataset('app/metrics/questions1.json')

from app.llm import print_prompt_and_response
val_queries_data = []
for query in val_data:
    retrieved_chunks = retrieve_context(query['question'], chunks, embedding_model, rag_index, print_rag_results=False, k_chunks=5)
    single_query_relevnt_data = print_prompt_and_response(query['question'], retrieved_chunks, model = llm_model_name,print_rag_results=True)
    print('===== Reference Correct Answer =====')
    precision, recall, f1, similarity = bertSimilarity(single_query_relevnt_data['llm_response'],query['answer'], 
                                                       tokenizer, model, scorer)
    evaluateTask(precision, recall, f1, similarity,task_type=2)
    print('===== Reference Wrong Answers =====')
    for wrong_answer in query['wrong_answers']:
        precision, recall, f1, similarity = bertSimilarity(single_query_relevnt_data['llm_response'],wrong_answer, 
                                                       tokenizer, model, scorer)
        evaluateTask(precision, recall, f1, similarity,task_type=2)

    print('='*150)
    # break
    



===== User Input Query =====
מהי תקופת הביטוח?
===== Relevant RAG Data =====
1 תקופת אכשרה-תקופה המתחילה במועד תחילת הביטוח.בקרות מקרה ביטוח במהלך תקופה זו,לא יהיה זכאי מבוטח
--------------------------------------------------
במהלך תקופת הכיסוי הרלוונטית.
--------------------------------------------------
מצב רפואי.
חברת הביטוח תשלם את ההוצאות בפועל וזאת עד לתקרה הקבועה בפוליסה.שים לב,במידה ויש לך כיסוי
--------------------------------------------------
סוג הביטוח כתב שירות
תקופת הביטוח מתחדשת כל שנתיים.ראה סעיף 10 לכתב השירות
--------------------------------------------------
לתגמולי ביטוח.
תיאור הכיסויים בפוליסה
שם הכיסוי תיאור הכיסוי מה הסכום המקסימלי שניתן לתבוע
--------------------------------------------------

===== Final LLM Response =====

תקופת הכיסוי עבור סוג הביטוח "כתב שירות" נמשכת כל שנתיים. ראה את הסעיף 10 של כתב השירות לקבלת גבולות התגמולים המקסימליים.========================================================================================================================